# Predicting the “Re-admission possibility of a patient into the hospital”

In [ ]:
#importing pandas, numpy
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt 

### loading train data

In [ ]:
TRAIN = pd.read_csv('Train.csv',header=0, encoding='UTF-8') #loading Train.csv

In [ ]:
#checking unique levels in age column
TRAIN['age'].unique()


array(['[80-90)', '[70-80)', '[60-70)', '[40-50)', '[90-100)', '[50-60)',
       '[30-40)', '[20-30)', '[10-20)', '[0-10)'], dtype=object)

In [ ]:
TRAIN_DIAGNOSIS = pd.read_csv('Train_Diagnosis_TreatmentData.csv',header=0, encoding='UTF-8') #loading Train_diagnosis.csv

In [ ]:
len(TRAIN_DIAGNOSIS['diabetesMed'].unique()) #total number of unique levels

2

In [ ]:
TRAIN_HOSPITALIZATION = pd.read_csv('Train_HospitalizationData.csv',header=0, encoding='UTF-8') #loading train_hospitalization

In [ ]:
TRAIN_HOSPITALIZATION.shape #checking the sape of train_hospitalization

(34650, 9)

In [ ]:
trdf = [TRAIN,TRAIN_HOSPITALIZATION,TRAIN_DIAGNOSIS] #making a combined dataframe of train.csv and train_hospitalization

In [ ]:
from functools import reduce  # import reduce from functools

#### final merged train data

In [ ]:
TRAIN_final = reduce(lambda left,right: pd.merge(left,right,on='patientID'), trdf) #reducting patientID to 1 from 2

In [ ]:
TRAIN_final.head() #checking the top 5 observations

,patientID,race,gender,age,weight,readmitted,AdmissionID,Admission_date,Discharge_date,admission_type_id,...,miglitol,troglitazone,tolazamide,insulin,glyburide.metformin,glipizide.metformin,metformin.rosiglitazone,metformin.pioglitazone,change,diabetesMed
0,PT11101,Caucasian,Male,[80-90),?,NO,ADM31792,2015-07-18,2015-07-23,2,...,No,No,No,Up,No,No,No,No,Ch,Yes
1,PT11102,Caucasian,Male,[70-80),?,NO,ADM42286,2016-04-23,2016-04-29,1,...,No,No,No,No,No,No,No,No,No,Yes
2,PT11103,?,Female,[60-70),?,NO,ADM43029,2016-05-13,2016-05-17,2,...,No,No,No,No,Up,No,No,No,Ch,Yes
3,PT11104,Caucasian,Female,[70-80),?,NO,ADM30585,2015-06-16,2015-06-18,1,...,No,No,No,No,No,No,No,No,No,Yes
4,PT11105,Caucasian,Female,[60-70),?,Within30days,ADM40516,2016-03-08,2016-03-09,1,...,No,No,No,Steady,No,No,No,No,No,Yes


In [ ]:
TRAIN_final.shape # finding the number of rows and columns in the final train dataset

(34650, 42)

In [ ]:
TRAIN_final.columns # columns in the final train dataset

Index(['race', 'gender', 'age', 'weight', 'Admission_date', 'Discharge_date',
       'admission_type_id', 'discharge_disposition_id', 'admission_source_id',
       'payer_code', 'medical_specialty', 'num_lab_procedures',
       'num_procedures', 'num_medications', 'num_diagnoses', 'diagnosis_1',
       'diagnosis_2', 'diagnosis_3', 'max_glu_serum', 'A1Cresult', 'metformin',
       'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride',
       'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'insulin', 'glyburide.metformin', 'glipizide.metformin',
       'metformin.rosiglitazone', 'metformin.pioglitazone', 'change',
       'diabetesMed'],
      dtype='object')

In [ ]:
TRAIN_final.corr() #correlation of the numerical train columns

,num_lab_procedures,num_procedures,num_medications,num_diagnoses
num_lab_procedures,1.000000,0.047882,0.257971,0.165134
num_procedures,0.047882,1.000000,0.412853,0.079586
num_medications,0.257971,0.412853,1.000000,0.254990
num_diagnoses,0.165134,0.079586,0.254990,1.000000


### loading test data

In [ ]:
TEST = pd.read_csv('Test.csv',header=0, encoding='UTF-8') # reading test.csv file

In [ ]:
TEST_HOSPITALIZATION = pd.read_csv('Test_HospitalizationData.csv',header=0, encoding='UTF-8') #reading test-hospitalization.csv

In [ ]:
TEST_DIAGNOSIS = pd.read_csv('Test_Diagnosis_TreatmentData.csv',header=0, encoding='UTF-8') #reading test_diagnosis.csv

In [ ]:
tsdf = [TEST,TEST_HOSPITALIZATION,TEST_DIAGNOSIS] # combined dataframe of test and test_hospitalization

### final merged test data

In [ ]:
TEST_final = reduce(lambda left,right: pd.merge(left,right,on='patientID'), tsdf) #combined test dataframe with diagnosis

In [ ]:
TEST_final.shape #shape of the test dataframe

(14630, 42)

In [ ]:
TRAIN_final=TRAIN_final.replace('?',np.NaN) #checking for nulls in train data

In [ ]:
TEST_final=TEST_final.replace('?',np.NaN) #checking for nulls in test data

In [ ]:
target = TRAIN_final.readmitted #defining our target

In [ ]:
target = pd.DataFrame(target) #dataframe of the target defined above

In [ ]:
TRAIN_final = TRAIN_final.drop(['readmitted'],axis=1) #dropping the target from train

### dropping AdmissionID and patientID

In [ ]:
TRAIN_final = TRAIN_final.drop(['AdmissionID','patientID'],axis = 1) 

In [ ]:
TEST_final = TEST_final.drop(['AdmissionID','patientID'],axis = 1) #

In [ ]:
# selecting columns to seperate categorical data
for col in ['race', 'gender', 'age', 'weight',
       'Admission_date', 'Discharge_date',  'payer_code',
       'medical_specialty', 'diagnosis_1', 'diagnosis_2',
       'diagnosis_3', 'max_glu_serum', 'A1Cresult', 'metformin', 'repaglinide',
       'nateglinide', 'chlorpropamide', 'glimepiride', 'acetohexamide',
       'glipizide', 'glyburide', 'tolbutamide', 'pioglitazone',
       'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone', 'tolazamide',
       'insulin', 'glyburide.metformin', 'glipizide.metformin',
       'metformin.rosiglitazone', 'metformin.pioglitazone', 'change',
       'diabetesMed', 'admission_type_id', 'discharge_disposition_id',
       'admission_source_id' ]:
    TRAIN_final[col] = TRAIN_final[col].astype('category')

In [ ]:
# selecting columns to seperate categorical data
for col in ['race', 'gender', 'age', 'weight', 'Admission_date', 'Discharge_date',
       'admission_type_id', 'discharge_disposition_id', 'admission_source_id',
       'payer_code', 'medical_specialty','diagnosis_1',
       'diagnosis_2', 'diagnosis_3', 'max_glu_serum', 'A1Cresult', 'metformin',
       'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride',
       'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'insulin', 'glyburide.metformin', 'glipizide.metformin',
       'metformin.rosiglitazone', 'metformin.pioglitazone', 'change',
       'diabetesMed']:
    TEST_final[col] = TEST_final[col].astype('category')

#### converting Admission_date and Discharge_date to datetime format

In [ ]:

TRAIN_final['Admission_date'] = pd.to_datetime(TRAIN_final['Admission_date'])
TRAIN_final['Discharge_date'] = pd.to_datetime(TRAIN_final['Discharge_date'])

In [ ]:
TEST_final['Admission_date'] = pd.to_datetime(TEST_final['Admission_date'])
TEST_final['Discharge_date'] = pd.to_datetime(TEST_final['Discharge_date'])

### finding the missing rows

In [ ]:
missing_cols = set( TRAIN_final.columns ) - set( TEST_final.columns )

In [ ]:
for c in missing_cols:
    TEST_final[c] = 0 

In [ ]:
TEST_final = TEST_final[TRAIN_final.columns] #making the number of variables same

#### dropping the columns and rows with more null values

In [ ]:
TRAIN_dataset = TRAIN_final.loc[:, TRAIN_final.isnull().mean() < .4] #dropping columns with more than 40% nulls
TEST_dataset = TEST_final.loc[:, TEST_final.isnull().mean() < .4]

In [ ]:
TRAIN_dataset = TRAIN_dataset.loc[TRAIN_dataset.isnull().mean(axis=1) < .3, :] #dropping rows with more than 30% nulls
TEST_dataset = TEST_dataset.loc[TEST_dataset.isnull().mean(axis=1) < .3, :]

In [ ]:
categorical = TRAIN_dataset.select_dtypes(include=['category']) #selecting categorical data from Train and making a new dataframe

In [ ]:
tcategorical = TEST_dataset.select_dtypes(include=['category']) #selecting categorical data from Test and making a new dataframe

In [ ]:
numerical = TRAIN_dataset.select_dtypes(include=['int64']) #selecting numerical data from Train and making a new dataframe

In [ ]:
tnumerical = TEST_dataset.select_dtypes(include=['int64']) #selecting numerical data from Test and making a new dataframe

#### Imputation

In [ ]:
categorical = categorical.apply(lambda x:x.fillna(x.value_counts().index[0])) #imputation for categorical train

In [ ]:
tcategorical = tcategorical.apply(lambda x:x.fillna(x.value_counts().index[0])) #imputation for categorical test

In [ ]:
numerical = numerical.fillna(numerical.mean()) #imputation for numerical train
tnumerical = tnumerical.fillna(numerical.mean()) #imputation for numerical test

#### dropping 'diagnosis_1','diagnosis_2','diagnosis_3','chlorpropamide','acetohexamide'

In [ ]:
train_categorical = categorical.drop(['diagnosis_1','diagnosis_2','diagnosis_3','chlorpropamide','acetohexamide','tolbutamide','miglitol','troglitazone','tolazamide','glipizide.metformin','metformin.rosiglitazone','metformin.pioglitazone',],axis=1)

In [ ]:
test_categorical = tcategorical.drop(['diagnosis_1','diagnosis_2','diagnosis_3','chlorpropamide','acetohexamide','tolbutamide','miglitol','troglitazone','tolazamide','glipizide.metformin','metformin.rosiglitazone','metformin.pioglitazone',],axis=1)

In [ ]:
from sklearn import preprocessing 

In [ ]:
standardized_num = preprocessing.scale(numerical) #scaling the data
standardized_tnum = preprocessing.scale(tnumerical)

In [ ]:
type(standardized_num)

numpy.ndarray

In [ ]:
standardized_num = pd.DataFrame(standardized_num)

In [ ]:
standardized_tnum = pd.DataFrame(standardized_tnum)

In [ ]:
TRAIN_CAT_DATA = pd.get_dummies(train_categorical) #creating dummies in train and test
TEST_CAT_DATA = pd.get_dummies(test_categorical)





In [ ]:
missing_cols_new = set( TRAIN_CAT_DATA.columns ) - set( TEST_CAT_DATA.columns ) # checking for columns imbalance

In [ ]:
for c in missing_cols_new:
    TEST_CAT_DATA[c] = 0

In [ ]:
TEST_CAT_DATA = TEST_CAT_DATA[TRAIN_CAT_DATA.columns] # removing for columns imbalance

In [ ]:
FINAL_TRAIN_DATA = pd.concat([TRAIN_CAT_DATA,standardized_num],axis=1) # concating scaled data and dummies
FINAL_TEST_DATA = pd.concat([TEST_CAT_DATA,standardized_tnum],axis=1)

In [ ]:
FINAL_TRAIN_DATA.drop(FINAL_TRAIN_DATA.columns[[123,124,125,126]], axis=1, inplace=True) # dropping columns 0,1,2,3

In [ ]:
FINAL_TEST_DATA.drop(FINAL_TEST_DATA.columns[[123,124,125,126]], axis=1, inplace=True) # dropping columns 0,1,2,3

In [ ]:
FINAL_TRAIN_DATA.shape ,  FINAL_TEST_DATA.shape #checking for equality of columns

((34650, 123), (14630, 123))

In [ ]:
FINAL_WITH_Y = pd.concat([FINAL_TRAIN_DATA,target],axis=1) #combining target with train

In [ ]:
cor=FINAL_WITH_Y.corr()

In [ ]:
cor= pd.DataFrame(cor)

In [ ]:
cor.to_csv("correlation.csv")

#### seperating train and test

In [ ]:
X = FINAL_WITH_Y.ix[:, FINAL_WITH_Y.columns != 'readmitted'] 
y = FINAL_WITH_Y.ix[:, FINAL_WITH_Y.columns == 'readmitted']

/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


In [ ]:
no = len(FINAL_WITH_Y[FINAL_WITH_Y.readmitted == 'Within30days'])

In [ ]:
with_days = np.array(FINAL_WITH_Y[FINAL_WITH_Y.readmitted == 'Within30days'].index)

In [ ]:
no_indices = FINAL_WITH_Y[FINAL_WITH_Y.readmitted == 'NO'].index

In [ ]:
random_normal_indices = np.random.choice(no_indices, no, replace = False)
random_normal_indices = np.array(random_normal_indices)

### undersampling

In [ ]:
under_sample_indices = np.concatenate([with_days,random_normal_indices])

In [ ]:
under_sample_data = FINAL_WITH_Y.iloc[under_sample_indices,:]

In [ ]:
under_sample_data.shape

(9518, 124)

In [ ]:
X_undersample = under_sample_data.ix[:, under_sample_data.columns != 'readmitted']

/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


In [ ]:
y_undersample = under_sample_data.ix[:, under_sample_data.columns == 'readmitted']

In [ ]:
#splitting train and validation
from sklearn.cross_validation import train_test_split




/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [ ]:
X_train_undersample, X_val_undersample, y_train_undersample, y_val_undersample = train_test_split(X_undersample
                                                                                                   ,y_undersample
                                                                                                   ,test_size = 0.1
                                                                                                   ,random_state = 0)

## Multinomial logistic for feature selection ACC to RFE

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics, cross_validation
from sklearn import datasets
from sklearn.feature_selection import RFE

In [ ]:
logreg = LogisticRegression() #

In [ ]:
rfe = RFE(logreg, 18)
rfe = rfe.fit(X_train_undersample, y_train_undersample ) # performing RFE for variable ranks
print(rfe.support_)
print(rfe.ranking_)

/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[False False False False False False False  True False False False False
 False False False False False False False False False False  True  True
 False  True False False False  True False False False False False  True
 False  True  True  True False False False False False  True  True False
  True False  True False False False False False False False False False
 False False False False False  True  True False False False False False
 False False False False False False False False False False False False
  True False False  True False False False False  True False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False]
[ 22  51  21  61  50  83  84   1   2  94  72  49  95  30  29  31  96  90
  91  27  18  28   1   1  74   1   8  23  88   1  64  62  19 104 101   1
   6   1   1   1 100  99  63 105  98   1   1  17   1   5   1  57  85   3
  33  26  24  60  48  59  11 10

In [ ]:
feature_names = list(X_train_undersample.columns.values) # list of variables

In [ ]:
print(sorted(zip(map(lambda x: round(x, 4), rfe.ranking_), feature_names))) # features sorted with respect to rank

[(1, 'admission_source_id_20'), (1, 'admission_source_id_22'), (1, 'admission_type_id_6'), (1, 'admission_type_id_7'), (1, 'age_[0-10)'), (1, 'discharge_disposition_id_1'), (1, 'discharge_disposition_id_11'), (1, 'discharge_disposition_id_13'), (1, 'discharge_disposition_id_14'), (1, 'discharge_disposition_id_15'), (1, 'discharge_disposition_id_22'), (1, 'discharge_disposition_id_23'), (1, 'discharge_disposition_id_25'), (1, 'discharge_disposition_id_28'), (1, 'discharge_disposition_id_5'), (1, 'glipizide_Down'), (1, 'nateglinide_Down'), (1, 'nateglinide_Up'), (2, 'age_[10-20)'), (3, 'admission_source_id_3'), (4, 'metformin_Up'), (5, 'discharge_disposition_id_27'), (6, 'discharge_disposition_id_12'), (7, 'diabetesMed_No'), (8, 'discharge_disposition_id_2'), (9, 'glyburide_Down'), (10, 'glyburide.metformin_Up'), (11, 'admission_source_id_10'), (12, 'admission_source_id_13'), (13, 'rosiglitazone_Down'), (14, 'nateglinide_No'), (15, 'nateglinide_Steady'), (16, 'repaglinide_No'), (17, 'dis

In [ ]:
#selecting the 1st 80 ranked features
cols = ["acarbose_Up", "admission_source_id_20", "age_[0-10)",  "age_[10-20)",  "discharge_disposition_id_11",  "discharge_disposition_id_13", "discharge_disposition_id_14",
        "discharge_disposition_id_15",  "discharge_disposition_id_2", "discharge_disposition_id_22", "discharge_disposition_id_28",  "discharge_disposition_id_3",
        "discharge_disposition_id_4", 
        "discharge_disposition_id_5", "discharge_disposition_id_9", "nateglinide_Down", "nateglinide_Up", "rosiglitazone_Up", "glipizide_No",  "glipizide_Steady", "admission_type_id_7", "discharge_disposition_id_8",
      "discharge_disposition_id_27", "discharge_disposition_id_12", "metformin_Up", "discharge_disposition_id_6", "glimepiride_Up", "race_Other", "rosiglitazone_Steady",
      "rosiglitazone_No", "admission_type_id_6", "diabetesMed_No", "glyburide.metformin_Up",  "admission_source_id_10", "admission_source_id_22",
     "discharge_disposition_id_16", "discharge_disposition_id_17", "race_Asian",  "pioglitazone_Down", "max_glu_serum_>300", "discharge_disposition_id_25",
     "discharge_disposition_id_23", "admission_source_id_6", "rosiglitazone_Down", "discharge_disposition_id_18", "admission_source_id_9",
     "glyburide_Down", "admission_source_id_3",  "admission_type_id_5",  "admission_type_id_3", "nateglinide_No",  "discharge_disposition_id_24",  "acarbose_No",
    "acarbose_Steady",  "glyburide.metformin_No", "discharge_disposition_id_19", "age_[70-80)",  "age_[80-90)",  "age_[30-40)", "age_[60-70)", "repaglinide_No",
    "glipizide_Down",  "nateglinide_Steady", "metformin_Down", "A1Cresult_Norm",  "pioglitazone_Steady",  "insulin_Down",  "admission_source_id_5",  "admission_type_id_8",
    "glimepiride_No", "metformin_Steady", "glimepiride_Down",  "repaglinide_Down",  "pioglitazone_Up",  "discharge_disposition_id_1",  "max_glu_serum_>200",  "max_glu_serum_None",
    "glyburide.metformin_Steady",  "race_Caucasian", "race_Hispanic", "race_AfricanAmerican",  "admission_source_id_4",  "diabetesMed_Yes", "glyburide_Steady",  "glyburide_No",  "glyburide_Up",  "admission_source_id_17",
    "A1Cresult_>8",  "gender_Female",  "gender_Male", "change_No", "age_[20-30)", "metformin_No", "change_Ch", "age_[40-50)", "max_glu_serum_Norm", "glipizide_Up",
        "admission_type_id_3","admission_type_id_5","A1Cresult_None","age_[20-30)","age_[90-100)","age_[50-60)","age_[30-40)","age_[90-100)", "age_[50-60)", "admission_type_id_4"]

In [ ]:
# new train data according to ranks

X_train_fin = X_train_undersample[cols]

In [ ]:
# new train target
y_train_fin=y_train_undersample

In [ ]:
# validation data
x_val = X_val_undersample[cols]

In [ ]:
#validataion target
y_val = y_val_undersample

In [ ]:
X=X_train_fin

In [ ]:
y=y_train_fin

In [ ]:
# using the same columns in test
FINAL_TEST = FINAL_TEST_DATA[cols]

## Model building

### Logistic 

In [ ]:

logreg = LogisticRegression()
logreg.fit(X, y) #fitting the model 

/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [ ]:
y_pred_val = logreg.predict(x_val) #predicting on validation data

In [ ]:
print(metrics.classification_report(y_val_undersample, y_pred_val)) # metrics for validation

              precision    recall  f1-score   support

          NO       0.62      0.64      0.63       479
Within30days       0.62      0.60      0.61       473

 avg / total       0.62      0.62      0.62       952



In [ ]:
#grid search on logistic

In [ ]:
# Create regularization penalty space
penalty = ['l1', 'l2']

# Create regularization hyperparameter space
C = np.logspace(0, 4, 10)

# Create hyperparameter options
hyperparameters = dict(C=C, penalty=penalty)

In [ ]:
clf_glm = GridSearchCV(logreg, hyperparameters, cv=5, verbose=0) #fitting the Gridsearch model
best_model = clf_glm.fit(X, y)

/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when 

/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when 

/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when 

/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when 

In [ ]:
# View best hyperparameters 
print('Best Penalty:', best_model.best_estimator_.get_params()['penalty'])
print('Best C:', best_model.best_estimator_.get_params()['C'])

Best Penalty: l2
Best C: 1.0


### RandomForest

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
    #randomForestClassifier
rfc = RandomForestClassifier(n_estimators=100, oob_score=True, random_state=123456)
rfc.fit(X, y)
    #Accuracy score
predicted = rfc.predict(FINAL_TEST) #predicting
pred_RFC= pd.DataFrame(predicted) #prediction dataframe
scores = cross_val_score(rfc,X, y,cv=10)  #cross validation
np.array(scores).mean() 

/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if sys.path[0] == '':
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversio

0.5601219751578534

In [ ]:
#performing grid search on Randim forest
from sklearn.model_selection import GridSearchCV
param_grid = { 
           "n_estimators" : [100,50,70,60,40,30,150,250],
           "max_depth" : [5,7,8,10,20,30,40,50,60,70,80,90,100],
           "min_samples_split" : [2,3,6, 8, 10,20,30,40,50]}
 
CV_rf = GridSearchCV(estimator=rfc, param_grid=param_grid)
CV_rf.fit(X, y) #fitting Grid CV model on data
print(CV_rfc.best_params_)

/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/model_s

/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/model_s

/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/model_s

/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/model_s

/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/model_s

...

/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/model_s

/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/model_s

/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/model_s

/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/model_s

NameError: name 'CV_rfc' is not defined

In [ ]:
print(CV_rf.best_params_) #best metrics for Random forest

{'max_depth': 5, 'min_samples_split': 30, 'n_estimators': 40}


In [ ]:
rfc = RandomForestClassifier(max_depth=5,min_samples_split = 30, n_estimators = 40 ,oob_score=True, random_state=123456)
rfc.fit(X, y) #fitting the best model
RFC_val_FI = rfc.predict(x_val)
print(metrics.classification_report(y_val, RFC_val_FI))



/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


              precision    recall  f1-score   support

          NO       0.62      0.62      0.62       479
Within30days       0.61      0.61      0.61       473

 avg / total       0.61      0.61      0.61       952



In [ ]:
rfc = RandomForestClassifier(max_depth=5,min_samples_split = 30, n_estimators = 40 ,oob_score=True, random_state=123456)
rfc.fit(X, y)
    #Accuracy score
predicted = rfc.predict(FINAL_TEST)
pred_RFC= pd.DataFrame(predicted)
    #pred_RFC.to_csv("pred_RFC.csv")
    #accuracy = accuracy_score(y_test, predicted)
    #print(accuracy)
    #Cross_val
scores = cross_val_score(rfc,X,y,cv=10)
np.array(scores).mean()

/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vec

0.5973599766627771

## Decision Tree Classifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X, y) #fitting the model on data and target

In [ ]:
# performing Gridsearch on Decisiontree classifier
from sklearn.model_selection import GridSearchCV


tree_para = {'criterion':['gini','entropy'],'max_depth':[1,2,3,4,5,6,7,8,9,10,11,12,15,20,30,40,50,70,90,120,150],
             'min_samples_split': [2,5,10,15,20,25,30,35,40,50,60],'min_samples_leaf': [1, 5, 10, 15, 20],
              'max_leaf_nodes': [None, 5, 10, 20]}
GridDt = GridSearchCV(DecisionTreeClassifier(), tree_para, cv=5)

In [ ]:
GridDt.fit(X,y) #fitting Grid model on data and target

GridSearchCV(cv=5, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'criterion': ['gini', 'entropy'], 'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 15, 20, 30, 40, 50, 70, 90, 120, 150], 'min_samples_split': [2, 5, 10, 15, 20, 25, 30, 35, 40, 50, 60], 'min_samples_leaf': [1, 5, 10, 15, 20], 'max_leaf_nodes': [None, 5, 10, 20]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [ ]:
print(GridDt.best_params_) #best parameters for decision tree classifier

{'criterion': 'entropy', 'max_depth': 9, 'max_leaf_nodes': None, 'min_samples_leaf': 1, 'min_samples_split': 35}


In [ ]:
# {'criterion': 'gini', 'max_depth': 3, 'max_leaf_nodes': 20, 'min_samples_leaf': 20, 'min_samples_split': 60}
#best so far
#'criterion': 'entropy', 'max_depth': 9, 'max_leaf_nodes': None, 'min_samples_leaf': 1, 'min_samples_split': 35
# Top
#clf_giniFI = DecisionTreeClassifier(criterion = "entropy",max_depth=8,min_samples_leaf = 1,min_samples_split = 130)

In [ ]:
clf_giniFI = DecisionTreeClassifier(criterion = "entropy",max_depth=8,min_samples_leaf = 1,min_samples_split = 130)

In [ ]:
clf_giniFI.fit(X, y) #fitting the final model

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=8,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=130,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [ ]:
#min_weight_fraction_leaf=0.0

In [ ]:
y_pred_val_giniFI = clf_giniFI.predict(x_val)



In [ ]:
print(metrics.classification_report(y_val, y_pred_val_giniFI))

              precision    recall  f1-score   support

          NO       0.64      0.45      0.52       479
Within30days       0.57      0.74      0.64       473

 avg / total       0.60      0.59      0.58       952



In [ ]:
y_pred_DT_entropy_best= clf_giniFI.predict(FINAL_TEST)

In [ ]:
y_pred_DT_entropy_best = pd.DataFrame(y_pred_DT_entropy_best)


In [ ]:
y_pred_DT_entropy_best.to_csv('FINAL_DT_FRIDAY2.csv',header=True, encoding='UTF-8')

### tweaking with parameters

In [ ]:
#clf_giniFII = DecisionTreeClassifier(criterion = "entropy",min_impurity_decrease=0.000,max_depth=7,min_samples_leaf = 10,min_samples_split = 150,min_weight_fraction_leaf=0.00107)

In [ ]:
clf_giniFII = DecisionTreeClassifier(criterion = "entropy",min_impurity_decrease=0.000,max_depth=7,min_samples_leaf = 10,random_state=None,presort=True,min_samples_split = 115,min_weight_fraction_leaf=0.00107)

In [ ]:
clf_giniFII.fit(X, y)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=7,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=10, min_samples_split=115,
            min_weight_fraction_leaf=0.00107, presort=True,
            random_state=None, splitter='best')

In [ ]:
y_pred_val_giniFII = clf_giniFII.predict(x_val)
print(metrics.classification_report(y_val, y_pred_val_giniFII))

              precision    recall  f1-score   support

          NO       0.65      0.42      0.51       479
Within30days       0.57      0.77      0.65       473

 avg / total       0.61      0.59      0.58       952



In [ ]:
y_pred_test_giniFII = clf_giniFII.predict(FINAL_TEST)
y_pred_test_giniFII = pd.DataFrame(y_pred_test_giniFII)
y_pred_test_giniFII.to_csv('FFF3.csv',header=True, encoding='UTF-8')



## Naive bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB
import numpy as np
NB = GaussianNB() # Naive Bayes 
NB.fit(X, y)  #Fitting Naive bayes model on test and train

In [ ]:
y_pred_NB= NB.predict(x_val)

In [ ]:
print(metrics.classification_report(y_val_undersample, y_pred_class))

## SVC

In [ ]:
from sklearn.svm import SVC
# Train model
clf_3 = SVC(kernel='rbf', 
            class_weight='balanced', # penalize
            probability=True)
 
clf_3.fit(X, y)



In [ ]:
SVC_pred_1 = clf_3.predict(x_val) # predict on validation

In [ ]:
SVC_pred_1= pd.DataFrame(SVC_pred_1)

print(metrics.classification_report(y_val_undersample, SVC_pred_1)) #metrics

In [ ]:
from sklearn import svm, grid_search
def svc_param_selection(X, y, nfolds):
    Cs = [0.001, 0.01, 0.1, 1, 10]
    gammas = [0.001, 0.01, 0.1, 1]
    param_grid = {'C': Cs, 'gamma' : gammas}
    grid_search = GridSearchCV(svm.SVC(kernel='rbf'), param_grid, cv=nfolds)
    grid_search.fit(X, y)
    grid_search.best_params_
    return grid_search.best_params_

In [ ]:
svc_param_selection(X, y, 5) #fitting grid searched best parameters

In [ ]:
clf_4 = SVC(C=1.0, kernel='rbf',gamma=0.1)
clf_4.fit(X, y)

In [ ]:
pred_clf_4 =clf_4.predict(x_val) 
print(metrics.classification_report(y_val_undersample, pred_clf_4))

#### stacked model

In [ ]:
#importing all the required packages
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.ensemble import RandomForestClassifier
from mlxtend.classifier import StackingClassifier
import numpy as np

In [ ]:
#new stacked model
clf1 = KNeighborsClassifier(n_neighbors=1) #using KNN
clf2 = RandomForestClassifier(random_state=1) #Random forest
clf3 = GaussianNB() #Naive bayes
lr = LogisticRegression() #Logistic
sclf = StackingClassifier(classifiers=[clf1, clf2, clf3], 
                          meta_classifier=lr)



In [ ]:
sclf #calling stacked model

StackingClassifier(average_probas=False,
          classifiers=[KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=1, p=2,
           weights='uniform'), RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto...
            oob_score=False, random_state=1, verbose=0, warm_start=False), GaussianNB(priors=None)],
          meta_classifier=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
          store_train_meta_features=False, use_clones=True,
          use_features_in_secondary=False, use_probas=False, verbose=0)

In [ ]:
y.readmitted = y.readmitted.replace("Within30days", 1) #converting target into 2d array for stacked model
y.readmitted = y.readmitted.replace("NO", 0)

/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [ ]:
#X_.readmitted = y.readmitted.replace("Within30days", 1)
#y.readmitted = y.readmitted.replace("NO", 0)

In [ ]:
for clf, label in zip([clf1, clf2, clf3, sclf], 
                      ['KNN', 
                       'Random Forest', 
                       'Naive Bayes',
                       'StackingClassifier']):

    scores = model_selection.cross_val_score(clf, X, y, 
                                              cv=3, scoring='accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" 
          % (scores.mean(), scores.std(), label))

/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


Accuracy: 0.53 (+/- 0.00) [KNN]


/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/utils/v

Accuracy: 0.55 (+/- 0.00) [Random Forest]
Accuracy: 0.52 (+/- 0.01) [Naive Bayes]


/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/mlxtend/classifier/stacking_classification.py:148: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(X, y)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/mlxtend/classifier/stacking_classification.py:148: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  clf.fit(X, y)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/mlxtend/classifier/stacking_classification.py:148: DataConversionWarning: A column-vector

Accuracy: 0.56 (+/- 0.00) [StackingClassifier]


In [ ]:
import matplotlib.pyplot as plt
from mlxtend.plotting import plot_decision_regions
import matplotlib.gridspec as gridspec
import itertools

In [ ]:
X= np.array(X)

In [ ]:
y=np.array(y)

In [ ]:
y

array([[1],
       [1],
       [0],
       ...,
       [0],
       [1],
       [1]])

In [ ]:
#running grid search for best parameters
gs = gridspec.GridSpec(2, 2)

#fig = plt.figure(figsize=(10,8))

for clf, lab, grd in zip([clf1, clf2, clf3, sclf], 
                         ['KNN', 
                          'Random Forest', 
                          'Naive Bayes',
                          'StackingClassifier'],
                          itertools.product([0, 1], repeat=2)):

    clf.fit(X, y)
    ax = plt.subplot(gs[grd[0], grd[1]])
    #fig = plot_decision_regions(X=X, y=y, clf=clf)
    plt.title(lab)

/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  if sys.path[0] == '':
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if sys.path[0] == '':
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/mlxtend/classifier/stacking_classification.py:148: DataConversionWarning: A column-vector y was passed when a 1d array was 

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from mlxtend.classifier import StackingClassifier

In [ ]:
# Initializing models

clf1 = KNeighborsClassifier(n_neighbors=1)
clf2 = RandomForestClassifier(random_state=1)
clf3 = GaussianNB()
lr = LogisticRegression()
sclf = StackingClassifier(classifiers=[clf1, clf2, clf3], 
                          meta_classifier=lr)

params = {'kneighborsclassifier__n_neighbors': [1, 5],
          'randomforestclassifier__n_estimators': [10, 50],
          'meta-logisticregression__C': [0.1, 10.0]}

In [ ]:
grid = GridSearchCV(estimator=sclf, 
                    param_grid=params, 
                    cv=5,
                    refit=True)
grid.fit(X, y)

/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/mlxtend/classifier/stacking_classification.py:148: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  clf.fit(X, y)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/mlxtend/classifier/stacking_classification.py:148: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(X, y)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/mlxtend/classifier/stacking_classification.py:148: DataConversionWarning: A column-vector

/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/mlxtend/classifier/stacking_classification.py:148: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  clf.fit(X, y)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/mlxtend/classifier/stacking_classification.py:148: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(X, y)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when

/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/mlxtend/classifier/stacking_classification.py:148: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  clf.fit(X, y)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/mlxtend/classifier/stacking_classification.py:148: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(X, y)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when

/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/mlxtend/classifier/stacking_classification.py:148: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  clf.fit(X, y)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/mlxtend/classifier/stacking_classification.py:148: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(X, y)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when

/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/mlxtend/classifier/stacking_classification.py:148: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  clf.fit(X, y)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/mlxtend/classifier/stacking_classification.py:148: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(X, y)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when

GridSearchCV(cv=5, error_score='raise',
       estimator=StackingClassifier(average_probas=False,
          classifiers=[KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=1, p=2,
           weights='uniform'), RandomForestClassifier(bootstrap=True, class_weight=None, cri...ures=False, use_clones=True,
          use_features_in_secondary=False, use_probas=False, verbose=0),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'kneighborsclassifier__n_neighbors': [1, 5], 'randomforestclassifier__n_estimators': [10, 50], 'meta-logisticregression__C': [0.1, 10.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [ ]:
print(grid.best_params_)

{'kneighborsclassifier__n_neighbors': 1, 'meta-logisticregression__C': 0.1, 'randomforestclassifier__n_estimators': 50}


In [ ]:
sclf2 = StackingClassifier(
          classifiers=[KNeighborsClassifier( n_neighbors=1 ), RandomForestClassifier(n_estimators=50)],
          meta_classifier=LogisticRegression(C=0.1))



In [ ]:
for clf, label in zip([clf1, clf2, sclf2], 
                      ['KNN', 
                       'Random Forest', 
                       'StackingClassifier']):

    scores = model_selection.cross_val_score(clf, X, y, 
                                              cv=3, scoring='recall')
    print("recall: %0.2f (+/- %0.2f) [%s]" 
          % (scores.mean(), scores.std(), label))

/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


recall: 0.53 (+/- 0.02) [KNN]


/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/mlxtend/classif

recall: 0.53 (+/- 0.02) [Random Forest]


/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/mlxtend/classifier/stacking_classification.py:148: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(X, y)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/mlxtend/classifier/stacking_classification.py:148: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  clf.fit(X, y)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/mlxtend/classifier/stacking_classification.py:148: DataConversionWarning: A column-vector

recall: 0.58 (+/- 0.01) [StackingClassifier]


In [ ]:
sclf2.fit(X,y)

/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/mlxtend/classifier/stacking_classification.py:148: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  clf.fit(X, y)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/mlxtend/classifier/stacking_classification.py:148: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(X, y)
/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


StackingClassifier(average_probas=False,
          classifiers=[KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=1, p=2,
           weights='uniform'), RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto... n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)],
          meta_classifier=LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
          store_train_meta_features=False, use_clones=True,
          use_features_in_secondary=False, use_probas=False, verbose=0)

In [ ]:
pred_stacked=sclf2.predict(x_val)

In [ ]:
pred_stacked

array([1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1,
       0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0,
       1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1,
       0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1,
       1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1,
       0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1,
       1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1,
       1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1,
       1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0,
       1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0,

In [ ]:
y_val_undersample.head()

,readmitted
1815,NO
9948,Within30days
19541,Within30days
22251,NO
18603,Within30days


In [ ]:
y_val.readmitted = y_val.readmitted.replace("Within30days", 1)
y_val.readmitted = y_val.readmitted.replace("NO", 0)

/Users/sandeepmoparthy/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [ ]:
yval= np.array(y_val)

In [ ]:
yval

array([[0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
    

In [ ]:
print(metrics.classification_report(yval, pred_stacked))

             precision    recall  f1-score   support

          0       0.55      0.52      0.54       479
          1       0.54      0.57      0.56       473

avg / total       0.55      0.55      0.55       952



In [ ]:
sclf3 = StackingClassifier(classifiers=[clf_giniFI, rfc, logreg], 
                          meta_classifier=lr)

In [ ]:
sclf3.fit(X,y)
pred_stacked2=sclf3.predict(x_val)

In [ ]:
print(metrics.classification_report(yval, pred_stacked2))

## thank you